<a href="https://colab.research.google.com/github/Serena-Anthony/vat-score-calculation/blob/main/vat_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Python libraries
!pip install --quiet \
    opencv-python-headless \
    moviepy \
    pytesseract \
    textstat \
    nltk \
    pydub \
    SpeechRecognition \
    ffmpeg-python \
    numpy \
    librosa \
    torch \
    torchvision \
    "httpx>=0.23.0,<1.0.0"

# Install system dependencies
!apt-get update -y
!apt-get install -y tesseract-ocr ffmpeg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
# ------------------------- 🧠 IMPORTS -------------------------
import librosa
import cv2
import pandas as pd
import numpy as np
import torch
from moviepy.editor import VideoFileClip
import torchvision.transforms as transforms
import torchvision.models as models
import tempfile

import pandas as pd
import numpy as np
import librosa
from moviepy.editor import VideoFileClip
import tempfile
import textstat

  if event.key is 'enter':



In [2]:

# ------------------------- 📁 CHOOSE INPUT MODE -------------------------
import os
from google.colab import drive
from google.colab import files

use_drive = input("Use Google Drive? (yes/no): ").strip().lower()

if use_drive == 'yes':
    drive.mount('/content/drive')
    base_path = input("Enter full path to your video folder in Drive (e.g., /content/drive/MyDrive/material): ").strip()
else:
    upload_path = "/content/uploaded_videos"
    os.makedirs(upload_path, exist_ok=True)
    uploaded = files.upload()
    for fname in uploaded.keys():
        os.rename(fname, os.path.join(upload_path, fname))
    base_path = upload_path

print(f"✅ Scanning folder: {base_path}")


Use Google Drive? (yes/no): no


Saving food_intro_1.mp4 to food_intro_1.mp4
✅ Scanning folder: /content/uploaded_videos


In [4]:
# ------------------------- 🎯 SCORING FUNCTIONS -------------------------

def get_visual_score(video_path):
    try:
        model = models.resnet18(pretrained=True)
        model.eval()

        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

        cap = cv2.VideoCapture(video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        scores = []
        frame_interval = max(1, frame_count // 10)

        for i in range(0, frame_count, frame_interval):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if ret:
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                input_tensor = transform(img).unsqueeze(0)
                with torch.no_grad():
                    output = model(input_tensor)
                    score = torch.nn.functional.softmax(output[0], dim=0).max().item()
                    scores.append(score)
        cap.release()
        return round(np.mean(scores) * 100, 2)
    except Exception as e:
        print(f"❌ Visual processing failed for {video_path}: {e}")
        return 0.0

def get_audio_score(video_path):
    try:
        with tempfile.NamedTemporaryFile(suffix=".wav") as tmp_audio:
            videoclip = VideoFileClip(video_path)
            audioclip = videoclip.audio
            audioclip.write_audiofile(tmp_audio.name, verbose=False, logger=None)
            y, sr = librosa.load(tmp_audio.name, sr=None)
            rms = librosa.feature.rms(y=y)
            score = np.mean(rms) * 1000  # Scaled RMS value
            return round(min(score, 100.0), 2)
    except Exception as e:
        print(f"❌ Audio processing failed for {video_path}: {e}")
        return 0.0

def get_textual_score(video_path):
    try:
        clip = VideoFileClip(video_path)
        duration = clip.duration
        fps = clip.fps
        frame_count = int(duration * fps)
        return round(min(frame_count / duration, 100.0), 2)  # proxy for text/slide density
    except Exception as e:
        print(f"❌ Textual processing failed for {video_path}: {e}")
        return 0.0


In [5]:
# ------------------------- 📊 PROCESS AND SCORE VIDEOS -------------------------
results = []

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".mp4"):
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, base_path)
            print(f"🎥 Processing video: {rel_path}")
            visual = get_visual_score(file_path)
            auditory = get_audio_score(file_path)
            textual = get_textual_score(file_path)
            results.append({
                "File": rel_path,
                "Visual": visual,
                "Auditory": auditory,
                "Textual": textual,
            })

# ------------------------- 📈 SHOW RESULTS -------------------------
df_scores = pd.DataFrame(results)
print("\n✅ Final Scores:")
print(df_scores)

  warnings.warn(

  warnings.warn(msg)



🎥 Processing video: food_intro_1.mp4


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 129MB/s]



✅ Final Scores:
               File  Visual  Auditory  Textual
0  food_intro_1.mp4    43.7     100.0     30.0
